# Modellierung Windeinspeisung

**Lernziele**
- Hyperparameteroptimierung mit tensorboard

## Pakete und Daten laden

In [1]:
# Datenorganisation
import pandas as pd
import numpy as np
import datetime 

# Ploterstellung
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib widget
import seaborn as sns

# Datenvorbereitung
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split

# Standardeinstellungen
plt.rcParams['axes.xmargin'] = 0
pd.set_option('display.precision',3)
np.set_printoptions(precision=3)

# Tensorflow
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Input, LSTM, Dense, Bidirectional, Dropout)
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import (EarlyStopping, ModelCheckpoint)
from tensorflow import keras

import tensorflow as tf
import keras_tuner as kt
import tensorboard
from tensorboard.plugins.hparams import api as hp
tensorboard.__version__
# Load the TensorBoard notebook extension
%load_ext tensorboard

#sonst  Module 
import copy
import sys
from math import ceil
import os 
from pathlib import Path

# Eingene Module 
# temporäre Einbinden des neuen Pfades mit zu ladenden Dateien
path = Path(sys.path[0])
sys.path.append(os.path.join(os.path.dirname(path.parent.parent.parent.absolute(),'00_internalfunctions'))
from KNN import KNN

SyntaxError: invalid syntax (1903405886.py, line 49)

In [17]:
# Laden des vorbereiteten WinddatenFrame
wind=pd.read_hdf('../daten/windeinspeisung_bereinigt','Obj1')

FileNotFoundError: File ../daten/windeinspeisung_bereinigt does not exist

## Datenorganisation

Erstellung eines KNN-Objektes mit normierten/Standardisierten Trainings/Validierungs- und Testdaten

In [ ]:
# Vorgaben zur Datentrennung
sections=(.6,.25,.15)
shuffle=False

# Vorbereitete Klasse
model = KNN()
# Unterteilung der Samples
model.split_sample(pd.DataFrame(wind["Ws_avg"]),pd.DataFrame(wind["S_avg"]),shuffle=shuffle,sections =sections)
#Auswahl der Skalierungsmethode
model.scaler("MinMax")

## Erstellung eines KNN zur Prognose der Einspeisung

**Definition Features und Output**

In [ ]:
model.numFeatures = 1
model.numResponses = 1;
model.x[0].shape

### Hyperparameter definieren

***HyperParameters (hp)***

Die Suche der Hyperparamters muss mit der Tuner API eingenständig definiert werden.<br>

Hierzu stehen 4 Variationsmöglichkeiten zur Verfügung:

*   `Boolean` —  Choice between True and False.<br>
     ***hp.Boolean***(name, default=False,...)    

*   `Choice` —  Choice of one value among a predefined set of possible values.<br>
     **hp.Choice**(
    name, values, ordered=None, default=None,...)

*   `Float` —  Floating point value hyperparameter.<br>
     ***hp.Float***(name,min_value,max_value,step=None,sampling="linear",default=None,...)
*   `Int` —  Integer point value hyperparameter.<br>
    ***hp.Int***('units', min_value=32, max_value=128, step=32,default=64)
*   `conditional_scope` —  Opens a scope to create conditional HyperParameters.<br> 

Die Ausprägungen der Parametervariation wird festgelegt.
hp.HParam ist eine Klasse, um die Hyperparameter zu definieren und zu loggen. Die Klasse wird von tensorboard zur Verfügung gestellt.

In [ ]:
HP_units = hp.HParam(name="units",domain = hp.IntInterval(min_value=32, max_value=212))     # Anzahl der Neuronen
HP_layers = hp.HParam(name="layers",domain = hp.IntInterval(min_value=0, max_value=2))      # Anzahl der hidden Layer 
HP_activation = hp.HParam(name="activation",domain=  hp.Discrete(["relu", "tanh"]))         # Aktivierungsmethode
HP_lr = hp.HParam(name = "lr", domain= hp.RealInterval(min_value=1e-4, max_value=1e-2))     # Lernrate
HP_dropout = hp.HParam('dropout', hp.RealInterval(0.1,0.6))                            # Dropout

### Modellaufbau mit Keras-functional API Schreibweise

Beim Aufruf von `single_run` wird das neuronale Netz mit den im aktuellem Lauf gültigen Hyperparameter-Set angelegt

In [ ]:
# Modellaufbau mit gewählter Hyperparameterausstattung

def single_run(hparams):
     
    # Define model layers.
    input_layer = Input(shape=(model.numFeatures,))
    
    X= Dense(hparams[HP_units],hparams[HP_activation],name='dense1')(input_layer)
    if layers >1:
        X = [Dense(hparams[HP_units],hparams[HP_activation],\
                             name='dense'+str(i+2))(X) for i in range(layers-1)]
  
        X = Dropout(hparams[HP_dropout])(X)
    
    # Output
    y_output= Dense(units='1', name='output')(X)  
    
    # Verbindung von Input und Output-Schicht   
    KNN = Model(inputs=input_layer,outputs=y_output)

    KNN.compile(optimizer=keras.optimizers.Adam(hparams[HP_lr]),
                loss=keras.losses.MeanSquaredError(),
                metrics=keras.losses.MeanAbsoluteError())
    
    return KNN


`run` ist eine Hilfsfunktion mit der ein neuer Lauf mit folgenden Ablauf erstellt wird:
1. Erstellung des neuronalen Netz mit übergebenen Hyperparametern `hparams`
2. Festlegen des Speicherorts für neue Logdatei
3. Durchführen des Trainings und speichern der Ergebnisse 

In [ ]:
def run(run_dir,hparams):
    KNN = single_run(hparams)
    early = EarlyStopping(monitor='val_loss', patience=5)

    KNN.fit(x=model.x_norm[0], 
          y=model.y_norm[0], 
          epochs=20,
          batch_size= 256, 
          validation_data=(model.x_norm[1], model.y_norm[1]), 
          callbacks=[
              early,
              tf.keras.callbacks.TensorBoard(run_dir),  # log metrics
              hp.KerasCallback(run_dir, hparams),       # log hparams
                    ],
       )

### Aufruf des Schleifenumlaufs 

1. Permutation aller Hyperparameter
2. Anstoss der Netzerstellung und des Trainings
3. Übermittlung von Informationen zum Stand des Umlaufs 

In [ ]:
session_num = 0

for units in (HP_units.domain.min_value, HP_units.domain.max_value):
    for layers in (HP_layers.domain.min_value, HP_layers.domain.max_value):
        for activation in HP_activation.domain.values:
            for lr in (HP_lr.domain.min_value, HP_lr.domain.max_value):
                for dropout in np.arange(HP_dropout.domain.min_value, HP_dropout.domain.max_value,.2):
                    hparams = {
                                HP_units: units,
                                HP_layers: layers,
                                HP_activation: activation,
                                HP_lr:lr,
                                HP_dropout:dropout
                              }
                    run_name = "run-%d" % session_num
                    print('--- Starting trial: %s' % run_name)
                    print({h.name: hparams[h] for h in hparams})
                    # start des nächsten Umlaufs mit veränderten Parametern
                    run('logs/hparam_tuning/' + run_name, hparams)
                    session_num += 1


Start TensorBoard through the command line or within a notebook experience. The two interfaces are generally the same. In notebooks, use the `%tensorboard` line magic. On the command line, run the same command without "%".

In [ ]:
%tensorboard --port 6031 --logdir logs/hparam_tuning

In [ ]:
#%tensorboard --logdir logs/runall

A brief overview of the visualizations created in this example and the dashboards (tabs in top navigation bar) where they can be found:

* **Scalars** show how the loss and metrics change with every epoch. You can use them to also track training speed, learning rate, and other scalar values. Scalars can be found in the **Time Series** or **Scalars** dashboards.
* **Graphs** help you visualize your model. In this case, the Keras graph of layers is shown which can help you ensure it is built correctly. Graphs can be found in the **Graphs** dashboard.
* **Histograms** and **Distributions** show the distribution of a Tensor over time. This can be useful to visualize weights and biases and verify that they are changing in an expected way. Histograms can be found in the **Time Series** or **Histograms** dashboards. Distributions can be found in the **Distributions** dashboard.

Additional TensorBoard dashboards are automatically enabled when you log other types of data. For example, the Keras TensorBoard callback lets you log images and embeddings as well. You can see what other dashboards are available in TensorBoard by clicking on the "inactive" dropdown towards the top right.



The left pane of the dashboard provides filtering capabilities that are active across all the views in the HParams dashboard:

    Filter which hyperparameters/metrics are shown in the dashboard
    Filter which hyperparameter/metrics values are shown in the dashboard
    Filter on run status (running, success, ...)
    Sort by hyperparameter/metric in the table view
    Number of session groups to show (useful for performance when there are many experiments)



he HParams dashboard has three different views, with various useful information:

    The Table View lists the runs, their hyperparameters, and their metrics.
    The Parallel Coordinates View shows each run as a line going through an axis for each hyperparemeter and metric. Click and drag the mouse on any axis to mark a region which will highlight only the runs that pass through it. This can be useful for identifying which groups of hyperparameters are most important. The axes themselves can be re-ordered by dragging them.
    The Scatter Plot View shows plots comparing each hyperparameter/metric with each metric. This can help identify correlations. Click and drag to select a region in a specific plot and highlight those sessions across the other plots.

A table row, a parallel coordinates line, and a scatter plot market can be clicked to see a plot of the metrics as a function of training steps for that session (although in this tutorial only one step is used for each run).